# Qaintellect training example 2: multiple qubits and measurement operators

In this example, we will demonstrate the optimisation of a quantum circuit with multiple qubits and measurement operators.

In [1]:
include("../src/Qaintellect.jl")
using .Qaintellect

using LinearAlgebra
using Flux
using IterTools: ncycle

We create a parametrized quantum circuit with $N$ qubits. We then optimize these parameters using the Flux framework.

In [2]:
# construct parametrized circuit
N = 4

ry = RyGate(√2)
n = randn(Float64, 3)
n /= norm(n)
rg = RotationGate(0.2π, n)
@show(rg)

cgc = CircuitGateChain{N}([
    single_qubit_circuit_gate(3, HadamardGate(), N),
    controlled_circuit_gate(2, (1, 4), RzGate(1.5π), N),
    two_qubit_circuit_gate(2, 3, SwapGate(), N),
    single_qubit_circuit_gate(3, PhaseShiftGate(0.3), N),
    single_qubit_circuit_gate(3, rg, N),
    single_qubit_circuit_gate(1, ry, N),
])
meas = MeasurementOps{N}([Matrix{Float64}(I, 2^N, 2^N), Hermitian(randn(ComplexF64, 2^N, 2^N))])

c = Circuit(cgc, meas)

rg = RotationGate([-0.17826508154831994, -0.3386457341072718, 0.4983219877901156])



    4 ————————•—————————————————————
              |                     
    3 —[H ]—————————x————[Pϕ]——[Rθ]—
              |     |               
    2 ———————[Rz]———x———————————————
              |                     
    1 ————————•————————————————[Ry]—


In [3]:
# set up model
model(ψ) = dot([0.3, -1.2], c(ψ))

# create loss function
loss(x, y) = Flux.mse(model(x), y)

# gather parameters from Circuit
paras = Flux.params(c)

# freeze parameter ry.θ, equivalent to delete!(paras, Qaintessent.get_trainable(ry))
delete!(paras, ry.θ)

paras

Params([Int64[], [4.71238898038469], Int64[], [0.3], Int64[], [-0.17826508154831994, -0.3386457341072718, 0.4983219877901156], Int64[], Int64[], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], Complex{Float64}[0.7209408138157799 + 0.0im -0.048187554593651934 - 0.785064470039137im … 0.6290119684156308 + 0.4814402088929597im 0.3781507411414241 - 0.3564995921757856im; -0.048187554593651934 + 0.785064470039137im -0.02650844542547566 + 0.0im … -0.04787686718017911 - 0.05827698614698754im 0.1008492417748543 + 0.09153977641313234im; … ; 0.6290119684156308 - 0.4814402088929597im -0.04787686718017911 + 0.05827698614698754im … 0.487293068235091 + 0.0im 0.775571408169586 - 0.7287085811159133im; 0.3781507411414241 + 0.3564995921757856im 0.1008492417748543 - 0.09153977641313234im … 0.775571408169586 + 0.7287085811159133im -1.397079234882013 + 0.0im]])

In [4]:
# input quantum state
ψ = randn(ComplexF64, 2^N)
ψ /= norm(ψ)

# desired output
e = 0.65

# set up data for training
data = ncycle([(ψ, e)], 64)

# define optimizer
opt = Descent(1)

# define evaluation function
evalcb() = @show(loss(ψ, e))

println("Initial model evaluation: $(model(ψ)), target: $e")

Flux.train!(loss, paras, data, opt, cb=Flux.throttle(evalcb, 0.01))

println("Final model evaluation: $(model(ψ)), target: $e")

Initial model evaluation: 1.1362015647193786, target: 0.65
loss(ψ, e) = 4.227001182507679
loss(ψ, e) = 0.07706347556022591
loss(ψ, e) = 0.0010005902814649009
loss(ψ, e) = 0.00014296227992032163
loss(ψ, e) = 1.797239052836422e-5
loss(ψ, e) = 5.802341772848861e-6
loss(ψ, e) = 1.8694320324694214e-6
loss(ψ, e) = 6.014152452801086e-7
loss(ψ, e) = 8.267932137359576e-8
loss(ψ, e) = 8.51117354882984e-9
loss(ψ, e) = 8.766501015716065e-10
Final model evaluation: 0.6500256845831587, target: 0.65


In [5]:
# check: Ry gate has not changed (frozen parameter)
@show(ry)

# on the other hand, parameters of general rotation gate are now different
@show(rg);

ry = RyGate([1.4142135623730951])
rg = RotationGate([-1.4032762538210892, 1.1163433920344525, 4.508883531707601])
